In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import resample
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# Define a list of stop words
stop_words = set(stopwords.words('english'))

# Load the Amazon reviews dataset
data_en = pd.read_pickle(r"C:\Users\A9236\Data\data_en.pickle")

# Combine the review headline and body into a single text column
data_en["Review"] = data_en["review_headline"] + " " + data_en["review_body"]

# Define the input and output data
X = data_en["Review"]
y = data_en["star_rating"]

print(data_en.shape)
print("X = ", X.shape)
print("y = ", y.shape)
data_en.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\A9236\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\A9236\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A9236\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(1753198, 18)
X =  (1753198,)
y =  (1753198,)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,language,processed_reviews,Review
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31,EN,use elit danger mac amaz joystick especi love ...,an amazing joystick. I especially love that yo...
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31,EN,love even realis game mous type silent mous se...,Definitely a silent mouse... Not a single clic...
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31,EN,poor qualiti advertis,One Star poor quality work and not as it is ad...
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31,EN,nice tend slip away stick intens hard press ga...,"good, but could be bettee nice, but tend to sl..."
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31,EN,great amiibo great collect qualiti materi desi...,"Great but flawed. Great amiibo, great for coll..."


In [2]:
# Upsample the minority classes to balance the dataset
rs = [resample(X[y == sr], y[y == sr], replace=False, n_samples=40000, random_state=123) for sr in [1,2,3,4,5]]
X_list = [rs[r][0] for r in range(5)]
y_list = [rs[r][1] for r in range(5)]
X_us = np.hstack(X_list)
y_us = np.hstack(y_list)

# Split the data into training, validation, and test sets
X_train, X_test_val, y_train, y_test_val = train_test_split(X_us, y_us, test_size= 0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, random_state=42)

print(X_us.shape)
print(y_us.shape)

(200000,)
(200000,)


In [3]:
# Tokenize the reviews into individual words and remove stop words
X_train_tokens = [[word for word in word_tokenize(review) if word.lower() not in stop_words] for review in X_train]
X_test_tokens = [[word for word in word_tokenize(review) if word.lower() not in stop_words] for review in X_test]

# Perform POS tagging on the tokenized reviews
X_train_pos = [pos_tag(tokens) for tokens in X_train_tokens]
X_test_pos = [pos_tag(tokens) for tokens in X_test_tokens]

TypeError: expected string or bytes-like object

In [37]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train word2vec model
w2v_model = Word2Vec(X_train_tokens, vector_size=100, window=5, min_count=5, workers=4)
#This will create a word2vec model with 100-dimensional vectors, a window size of 5

# Create feature vectors
def create_features(review_tokens, model):
    num_features = model.vector_size
    features = np.zeros((len(review_tokens), num_features), dtype="float32")
    for i, tokens in enumerate(review_tokens):
        for token in tokens:
            if token in model.wv.key_to_index:
                features[i] += model.wv.get_vector(token)
    return features

X_train_features = create_features(X_train_tokens, w2v_model)
X_test_features = create_features(X_test_tokens, w2v_model)

# Train logistic regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_features, y_train)

# Make predictions on test set
y_pred = lr_model.predict(X_test_features)

# Evaluate model performance
print(classification_report(y_test, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [119997, 120000]

Trying with 200k Lines takes very long for my computer. Maybe we try on someone elses? Also I did not integrate our stop-words yet and could also try a version with -1 (1-2 stars), 0 (3stars) and 1 (4-5 stars) as target variable.. accuracy with 10k lines was at around 60% already, so I think it is a step in the right direction considering LR is a rather simple model for this task.

In [ ]:
#data_en.to_csv("path", compression='zip')